In [4]:
import nltk
import tensorflow
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import preprocessor as p
import re
import string

import pandas as pd
df=pd.read_csv("tweet_emotions.csv")

In [2]:
!pip install preprocessor

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for preprocessor: filename=preprocessor-1.1.3-py3-none-any.whl size=4473 sha256=a27730e2223f5afe001ed9db24c03db2908ee9909163b89ea5d163565982e31d
  Stored in directory: /root/.cache/pip/wheels/b6/26/c1/ce3fdf93e63c11019da18dbeec4c09309d3edc5461c2a5fffb
Successfully built preprocessor


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tweet_id   40000 non-null  int64 
 1   sentiment  40000 non-null  object
 2   content    40000 non-null  object
dtypes: int64(1), object(2)
memory usage: 937.6+ KB


In [9]:
df.drop('tweet_id',axis=1,inplace=True)

In [10]:
len(df)

40000

In [12]:
df

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...
...,...,...
39995,neutral,@JohnLloydTaylor
39996,love,Happy Mothers Day All my love
39997,love,Happy Mother's Day to all the mommies out ther...
39998,happiness,@niariley WASSUP BEAUTIFUL!!! FOLLOW ME!! PEE...


In [11]:
df.shape

(40000, 2)

In [13]:
def emotions(df):
    df.replace(to_replace=['love','surprise','happiness','fun','enthusiasm'],value = 1,inplace = True)
    df.replace(to_replace=['neutral'],value = 0,inplace = True)
    df.replace(to_replace=['anger','sadness','empty','worry','relief','hate','boredom'],value = -1,inplace = True)
    return df

In [14]:
df['sentiment']=emotions(df['sentiment'])

In [15]:
df['sentiment'].value_counts()

-1    17589
 1    13773
 0     8638
Name: sentiment, dtype: int64

In [16]:
import re
def preprocess_text(text):
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    text = re.sub(r'\S+@\S+', '', text)
    tokens = word_tokenize(text)
    tokens = [token.lower() for token in tokens]
    tokens = [token for token in tokens if token not in string.punctuation]
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text


In [17]:
df['preprocessed_text']=df['content'].apply(preprocess_text)
preprocessed_text=df['preprocessed_text'].values

In [5]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [7]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [18]:
preprocessed_text.tolist()

['tiffanylue i know i wa listenin to bad habit earlier and i started freakin at his part',
 'layin n bed with a headache ughhhh ... waitin on your call ...',
 'funeral ceremony ... gloomy friday ...',
 'want to hang out with friend soon',
 'dannycastillo we want to trade with someone who ha houston ticket but no one will',
 "re-pinging ghostridah14 why did n't you go to prom bc my bf did n't like my friend",
 "i should be sleep but im not thinking about an old friend who i want but he 's married now damn amp he want me 2 scandalous",
 'hmmm is down',
 'charviray charlene my love i miss you',
 "kelcouch i 'm sorry at least it 's friday",
 'cant fall asleep',
 'choked on her retainer',
 'ugh i have to beat this stupid song to get to the next rude',
 'brodyjenner if u watch the hill in london u will realise what tourture it is because were week and week late i just watch itonlinelol',
 'got the news',
 'the storm is here and the electricity is gone',
 'annarosekerr agreed',
 "so sleepy ag

In [19]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [20]:
vectorizer = CountVectorizer(ngram_range=(1,2))
x=vectorizer.fit_transform(preprocessed_text)

In [21]:
x

<40000x260923 sparse matrix of type '<class 'numpy.int64'>'
	with 916530 stored elements in Compressed Sparse Row format>

In [22]:
import numpy as np
y=df['sentiment'].values
y[39995]

0

In [23]:
from sklearn.model_selection import train_test_split

In [24]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.35, random_state=42)

In [25]:
from sklearn.metrics import accuracy_score,classification_report

In [26]:
#DECISION TREE CLASSIFIER
from sklearn.tree import DecisionTreeClassifier
classifier=DecisionTreeClassifier()
classifier.fit(xtrain, ytrain)
ypred = classifier.predict(xtest)
accuracy = accuracy_score(ytest, ypred)
print("Accuracy:", accuracy)

Accuracy: 0.5137857142857143


In [27]:
print(classification_report(ytest,ypred))

              precision    recall  f1-score   support

          -1       0.57      0.61      0.59      6170
           0       0.36      0.32      0.34      3062
           1       0.53      0.52      0.52      4768

    accuracy                           0.51     14000
   macro avg       0.48      0.48      0.48     14000
weighted avg       0.51      0.51      0.51     14000



In [28]:
#NAIVE BAYES
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(xtrain, ytrain)
ypred = classifier.predict(xtest)
accuracy = accuracy_score(ytest, ypred)
print("Accuracy:", accuracy)

Accuracy: 0.5720714285714286


In [29]:
print(classification_report(ytest,ypred))

              precision    recall  f1-score   support

          -1       0.57      0.82      0.67      6170
           0       0.44      0.07      0.12      3062
           1       0.59      0.57      0.58      4768

    accuracy                           0.57     14000
   macro avg       0.53      0.49      0.46     14000
weighted avg       0.55      0.57      0.52     14000



In [30]:
#RANDOM FOREST
from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier()
classifier.fit(xtrain, ytrain)
predictions = classifier.predict(xtest)
accuracy = accuracy_score(ytest, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.5712142857142857


In [31]:
print(classification_report(ytest,ypred))

              precision    recall  f1-score   support

          -1       0.57      0.82      0.67      6170
           0       0.44      0.07      0.12      3062
           1       0.59      0.57      0.58      4768

    accuracy                           0.57     14000
   macro avg       0.53      0.49      0.46     14000
weighted avg       0.55      0.57      0.52     14000

